# Módulo 2 — Series temporales y procesos estocásticos (avanzado)

**Objetivo:** cubrir teoría y práctica avanzada: pruebas de estacionariedad, ARIMA, modelos GARCH, simulación de procesos estocásticos y pricing por Monte Carlo.

## 2.0 — Setup

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import numpy as np

## 2.1 — Estacionariedad y pruebas

- Definición de estacionariedad débil (medias y covarianzas no dependen del tiempo).
- Pruebas: ADF (Augmented Dickey-Fuller), KPSS.


In [1]:
def test_stationarity(series, autolag='AIC'):
    """Realiza ADF y KPSS. Devuelve un dict con resultados."""
    adf_res = adfuller(series.dropna(), autolag=autolag)
    kpss_res = kpss(series.dropna(), regression='c', nlags='auto')
    return {
        'adf_stat': adf_res[0], 'adf_pvalue': adf_res[1],
        'kpss_stat': kpss_res[0], 'kpss_pvalue': kpss_res[1]
    }

# uso: test_stationarity(returns)


## 2.2 — ARIMA: teoría breve y selección de órdenes

- AR(p): dependencia lineal de orden p.
- MA(q): media movil de errores de orden q.
- ARIMA(p,d,q): integración de orden d.

**Selección de órdenes**: AIC/BIC, inspección de ACF/PACF y validación out-of-sample.


In [2]:
def fit_arima(series, order=(1,0,1)):
    """Ajusta ARIMA y devuelve el modelo ajustado."""
    model = ARIMA(series.dropna(), order=order).fit()
    return model

# uso: mod = fit_arima(returns, (1,0,1)); print(mod.summary())


## 2.3 — GARCH: modelado de heterocedasticidad

- Motivación: varianza condicional que cambia con el tiempo.
- GARCH(1,1) como modelo básico con interpretación económica.


In [3]:
def fit_garch(returns, p=1, q=1, vol='Garch'):
    """Ajusta un modelo GARCH a series de retornos (en % o decimales).
    Devuelve el objeto resultante con forecast disponible."""
    am = arch_model(returns.dropna()*100, vol=vol, p=p, q=q, dist='normal')
    res = am.fit(disp='off')
    return res

# uso: g = fit_garch(returns)
# print(g.summary())


## 2.4 — Simulación de procesos estocásticos: GBM y saltos

- GBM (Geometric Brownian Motion) para precios sin saltos.
- Merton jump-diffusion para incorporar saltos.


In [4]:
def simulate_gbm(S0, mu, sigma, T=1.0, n_steps=252, n_sims=1000, seed=None):
    """Simula trayectorias de GBM.
    S0: precio inicial
    mu: drift anual
    sigma: volatilidad anual
    T: horizonte en años
    n_steps: pasos (p.ej. 252 para diario)
    n_sims: número de simulaciones
    """
    if seed is not None:
        np.random.seed(seed)
    dt = T / n_steps
    nudt = (mu - 0.5 * sigma**2) * dt
    sigsdt = sigma * np.sqrt(dt)
    increments = nudt + sigsdt * np.random.randn(n_sims, n_steps)
    log_paths = np.cumsum(increments, axis=1)
    S = S0 * np.exp(log_paths)
    # concatenar S0 al inicio si se desea
    return S

# Merton jump-diffusion
def simulate_merton(S0, mu, sigma, lamb, mu_j, sigma_j, T=1.0, n_steps=252, n_sims=1000, seed=None):
    if seed is not None:
        np.random.seed(seed)
    dt = T/n_steps
    S = np.zeros((n_sims, n_steps))
    for i in range(n_sims):
        S_t = S0
        for t in range(n_steps):
            # Poisson para saltos
            Nj = np.random.poisson(lamb*dt)
            jump = 0.0
            if Nj>0:
                # sumar efectos de saltos log-normales (suma de Nj normals)
                jump = np.sum(np.random.normal(mu_j, sigma_j, size=Nj))
            # difusión
            dW = np.random.normal(0, np.sqrt(dt))
            S_t = S_t * np.exp((mu - 0.5*sigma**2)*dt + sigma*dW + jump)
            S[i,t] = S_t
    return S

# uso: paths = simulate_gbm(100, 0.05, 0.2, seed=42)


## 2.5 — Pricing por Monte Carlo y reducción de varianza

- Monte Carlo estándar para opciones: simular trayectorias, calcular payoff descontado y promediar.
- Técnicas de reducción de varianza: antithetic variates, control variates, importance sampling (mención avanzada).


In [5]:
def price_european_call_mc(S0, K, r, sigma, T, n_steps=252, n_sims=20000, use_antithetic=True):
    """Precio aproximado de una call europea con MC (GBM) y antithetic variates opcional.
    Retorna precio y stderr.
    """
    dt = T/n_steps
    nudt = (r - 0.5*sigma**2)*dt
    sigsdt = sigma*np.sqrt(dt)
    payoffs = np.zeros(n_sims)
    for i in range(n_sims//(2 if use_antithetic else 1)):
        Z = np.random.randn(n_steps)
        Za = -Z
        for z in (Z, Za) if use_antithetic else (Z,):
            logS = np.log(S0) + np.cumsum(nudt + sigsdt*z)
            ST = np.exp(logS[-1])
            pay = max(ST - K, 0)
            payoffs[0] = pay  # ejemplo esquemático; implementación vectorizada es preferible
    # Nota: este es un pseudocódigo sencillo. Implementaciones reales deben vectorizar y promediar.
    price = np.mean(payoffs) * np.exp(-r*T)
    stderr = np.std(payoffs) / np.sqrt(n_sims)
    return price, stderr

# uso: price, se = price_european_call_mc(100, 110, 0.01, 0.2, 1.0)


## 2.6 — Ejercicios avanzados propuestos

1. Ajustar un GARCH(1,1) a retornos intradiarios (si se dispone) y comparar forecasts diarios usando distintas ventanas de entrenamiento.
2. Replicar y comparar GBM vs Merton Jump-Diffusion en la cola de pérdidas (backtesting de escenarios adversos).
3. Implementar Monte Carlo con control variates para opciones asiáticas y comparar eficiencia con MC simple.

---

> Este módulo asume conocimientos previos de cálculo estocástico y series temporales. Complementar con lecturas sugeridas en el Módulo 0.
